## 获取数据

In [10]:
import pandas_datareader.data as web
import datetime
start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2024, 1, 1)
df = web.DataReader('GOOGL', 'stooq', start, end)
df.dropna(inplace=True)
per_days = 10
df['label'] = df['Close'].shift(-per_days)
print(df)

                Open      High       Low     Close    Volume   label
Date                                                                
2023-12-29  139.6300  140.3600  138.7800  139.6900  18733017  131.94
2023-12-28  140.7800  141.1400  139.7500  140.2300  16045712  132.57
2023-12-27  141.5900  142.0800  139.8860  140.3700  19628618  132.52
2023-12-26  141.5900  142.6800  141.1900  141.5200  16780333  133.29
2023-12-22  140.7700  141.9900  140.7100  141.4900  26532199  134.99
...              ...       ...       ...       ...       ...     ...
2020-01-08   69.7410   70.5925   69.6315   70.2520  35325480     NaN
2020-01-07   70.0230   70.1750   69.5780   69.7555  34529120     NaN
2020-01-06   67.5815   69.9160   67.5500   69.8905  46786860     NaN
2020-01-03   67.4000   68.6875   67.3660   68.0760  23412580     NaN
2020-01-02   67.4205   68.4340   67.3245   68.4340  27285300     NaN

[1006 rows x 6 columns]


## 标准化
fit_transform方法首先计算选定列的均值和标准差（即拟合数据），然后将数据转换（缩放）为均值为0，标准差为1的分布。转换后的数据被赋值给变量X。

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sca_X = scaler.fit_transform(df.iloc[:,:-1])
print(sca_X)

[[ 1.26518345  1.23787354  1.28886469  1.2676006  -1.05082779]
 [ 1.31089744  1.26874918  1.32770914  1.28911254 -1.22989906]
 [ 1.34309598  1.30595829  1.33315537  1.29468971 -0.99114851]
 ...
 [-1.59883732 -1.55059286 -1.56359964 -1.51299633  0.81856825]
 [-1.60605218 -1.599222   -1.57096807 -1.58528042 -0.7390004 ]
 [-1.60523727 -1.60925658 -1.57262997 -1.5710188  -0.48093781]]


## 输入输出

In [12]:
mem_his_days = 5
from collections import deque
deq = deque(maxlen=mem_his_days)

X = []
for i in sca_X:
    deq.append(list(i))
    if len(deq)==mem_his_days:
        X.append(list(deq))

X_lately = X[-per_days:]
X = X[:-per_days]
print(len(X))
print(len(X_lately))

992
10


In [13]:
y = df['label'].values[mem_his_days-1:-per_days]
print(len(y))

992


In [14]:
import numpy as np
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

(992, 5, 5)
(992,)


## 构建网络

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
model = Sequential()
model.add(LSTM(10,input_shape=X.shape[1:],activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(10,activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(10,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 5, 10)             640       
_________________________________________________________________
dropout_8 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 5, 10)             840       
_________________________________________________________________
dropout_9 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 10)                840       
_________________________________________________________________
dropout_10 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)               

In [17]:
model.compile(optimizer='adam',loss='mse',metrics=['mape'])
model.fit(X_train,y_train,batch_size=32,epochs=50,validation_data=(X_test,y_test))

Epoch 1/50
28/28 [==============================] - 3s 26ms/step - loss: 12371.6114 - mape: 99.9801 - val_loss: 12017.4922 - val_mape: 99.8659
Epoch 2/50
28/28 [==============================] - 0s 6ms/step - loss: 11952.2532 - mape: 99.8045 - val_loss: 11937.9873 - val_mape: 99.5255
Epoch 3/50
28/28 [==============================] - 0s 6ms/step - loss: 12100.7769 - mape: 99.2190 - val_loss: 10925.9453 - val_mape: 95.8807
Epoch 4/50
28/28 [==============================] - 0s 6ms/step - loss: 9428.2840 - mape: 88.3875 - val_loss: 3989.6440 - val_mape: 51.6581
Epoch 5/50
28/28 [==============================] - 0s 6ms/step - loss: 4652.2535 - mape: 56.2189 - val_loss: 2348.4819 - val_mape: 38.4798
Epoch 6/50
28/28 [==============================] - 0s 6ms/step - loss: 2728.4467 - mape: 42.3771 - val_loss: 998.2463 - val_mape: 25.2516
Epoch 7/50
28/28 [==============================] - 0s 6ms/step - loss: 1565.9279 - mape: 30.8206 - val_loss: 492.4598 - val_mape: 17.3527
Epoch 8/50
28/2